In [3]:
import os
sym = 'BTCUSDT'
os.path.join('data/ohlcv', f"{sym}.pickle")

'data/ohlcv\\BTCUSDT.pickle'

In [6]:
from lib import market as M, features as F, utils as U

In [2]:
import os, json, numpy as np, pandas as pd
from config import Paths, NewsCfg, MarketCfg, FeatureCfg
from lib import news as N, market as M, features as F, utils as U


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\hr_ba\anaconda3\envs\hftlob\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\hr_ba\anaconda3\envs\hftlob\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\hr_ba\anaconda3\envs\hftlob\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\hr_ba\anaconda3\envs\hftlob\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start(

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [ ]:
U.set_seed(123)
P = Paths(); NC = NewsCfg(); MC = MarketCfg(); FC = FeatureCfg()

tradable = MC.symbols_usdt
news_dir = os.path.join(P.data_dir, "raw_news")
csvs = [os.path.join(news_dir, f) for f in os.listdir(news_dir) if f.endswith(".csv")]
if not csvs:
    raise FileNotFoundError(f"No CSVs found in {news_dir}. Place your news CSVs there.")

df_news = N.read_and_onehot_news(csvs, tradable)

if os.path.exists(P.news_pickle):
    df_news = pd.read_pickle(P.news_pickle)
    print("Loaded cached news embeddings.")
    no_news_vec = df_news["embedding"].iloc[0] * 0
else:
    tok, mdl, device, max_len = N.load_text_encoder(NC.model_path)
    emb = N.embed_texts(df_news["content"].tolist(), tok, mdl, device, max_len, NC.pooling, NC.batch_size)
    df_news["embedding"] = [e for e in emb]
    no_news_vec = N.embed_texts([NC.no_news_token], tok, mdl, device, max_len, NC.pooling, batch_size=1)[0]
    df_news["news_count"] = 1
    os.makedirs(os.path.dirname(P.news_pickle), exist_ok=True)
    df_news.to_pickle(P.news_pickle)
    print("Saved news embeddings.")

news_3m = N.resample_news_3m(df_news[["releasedAt","embedding","news_count"] +
                        [s.replace("USDT","") for s in tradable if s.endswith("USDT")]].copy(),
                        np.asarray(no_news_vec), rule=NC.rule)

client = M.get_client("", "")
per_asset = []
for sym in MC.symbols_usdt:
    fpath = os.path.join(P.ohlcv_dir, f"{sym}.pickle")
    if os.path.exists(fpath):
        df1m = pd.read_pickle(fpath)
    else:
        df1m = M.fetch_klines(client, sym, MC.timeframe, MC.start_date, MC.end_date)
        M.cache_symbol(df1m, fpath)
    df3m = M.resample_to_3m(df1m, FC.agg_cols)
    per_asset.append(F.add_targets_and_features(df3m, FC.fwd_h, FC.seq_len, FC.per_asset_features, sym))

merged = F.merge_assets(per_asset)
merged = F.attach_news(merged, news_3m)

tcols = F.make_time_cols(merged)
merged = pd.concat([merged, tcols], axis=1)
os.makedirs(os.path.dirname(P.merged_parquet), exist_ok=True)
merged.to_parquet(P.merged_parquet, index=False)
print("Saved merged 3m parquet.")

tr_days, va_days, te_days = F.train_val_test_by_day(merged)
with open(P.splits_json, "w") as f: json.dump({"train": [str(x) for x in tr_days],
                                                "val": [str(x) for x in va_days],
                                                "test": [str(x) for x in te_days]}, f)

all_cols = [c for c in merged.columns if any(c.endswith(x) for x in ["close","volume","numberOfTrades",
                                                                        "prev_return","prev_volatility","return","volatility"])]
target_cols = [c for c in merged.columns if c.endswith("_target_return")]
feat_cols = [c for c in all_cols if (("return" not in c) or ("prev_return" in c))]
feat_cols = [c for c in feat_cols if (("volatility" not in c) or ("prev_volatility" in c))]

count_cols = [c for c in merged.columns if c in [s[:-4] for s in MC.symbols_usdt]]

df_tr, df_va, df_te, stats = F.normalize_train_val_test(merged.fillna(0),
                                                        feature_cols=feat_cols,
                                                        split_days=(tr_days, va_days, te_days),
                                                        save_path=P.normalizer_pkl)
df_tr.to_parquet(P.processed_dir + "/train.parquet", index=False)
df_va.to_parquet(P.processed_dir + "/val.parquet", index=False)
df_te.to_parquet(P.processed_dir + "/test.parquet", index=False)
with open(P.processed_dir + "/meta.json","w") as f:
    json.dump({"feature_cols":feat_cols, "target_cols":target_cols, "count_cols":count_cols}, f)


In [ ]:



def main():

if __name__ == "__main__":
    main()


c:\Users\hr_ba\anaconda3\envs\hftlob\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Error importing huggingface_hub.hf_api: cannot import name 'ObjectWrapper' from 'tqdm.utils' (c:\Users\hr_ba\anaconda3\envs\hftlob\lib\site-packages\tqdm\utils.py)
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\hr_ba\anaconda3\envs\hftlob\lib\site-packages\IPython\core\interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\hr_ba\AppData\Local\Temp\ipykernel_28984\110170634.py", line 3, in <module>
    from lib import news as N, market as M, features as F, utils as U
  File "g:\NovoExpert\Gitlab_portfolio\portfolio 4h (long&short)\Joint distribution\joint_portfolio\lib\news.py", line 4, in <module>
    from transformers import AutoTokenizer, AutoModel
  File "c:\Users\hr_ba\anaconda3\envs\hftlob\lib\site-packages\transformers\__init__.py", line 27, in <module>
    from . import dependency_versions_check
  File "c:\Users\hr_ba\anaconda3\envs\hftlob\lib\site-packages\transformers\dependency_versions_check.py", line 16, in <module>
    from .utils.versions import require_version, require_version_core
  File "c:\Users\hr_ba\anaconda3\envs\hftlob\lib\site-packages\transformers\utils\__in

In [2]:
!pip install transformers

     ---------------------------------------- 11.6/11.6 MB 3.4 MB/s eta 0:00:00
     -------------------------------------- 320.2/320.2 kB 5.0 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 4.7 MB/s eta 0:00:00


     ------------------------------------- 561.5/561.5 kB 17.8 MB/s eta 0:00:00
     ---------------------------------------- 78.5/78.5 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.36.1
    Uninstalling tqdm-4.36.1:
      Successfully uninstalled tqdm-4.36.1
